# Connect to your workspace
[docs](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python)


In [1]:
from azureml.core import Workspace, Dataset
ws = Workspace.from_config()

In [2]:
import urllib.request
from azureml.core.model import Model

# Download model
urllib.request.urlretrieve("https://aka.ms/bidaf-9-model", "model.onnx")

# Register model
model = Model.register(ws, model_name="bidaf_onnx", model_path="./model.onnx")

Registering model bidaf_onnx


In [3]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="project_environment")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory="../",
    entry_script="./score.py",
)

In [4]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [5]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    dummy_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Generating Docker build context.
2021/09/24 14:03:26 Downloading source code...
2021/09/24 14:03:27 Finished downloading source code
2021/09/24 14:03:27 Creating Docker network: acb_default_network, driver: 'bridge'
2021/09/24 14:03:28 Successfully set up Docker network: acb_default_network
2021/09/24 14:03:28 Setting up Docker configuration...
2021/09/24 14:03:28 Successfully set up Docker configuration
2021/09/24 14:03:28 Logging in to registry: acrabstractwe.azurecr.io
2021/09/24 14:03:30 Successfully logged into acrabstractwe.azurecr.io
2021/09/24 14:03:30 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/09/24 14:03:30 Scanning for dependencies...
2021/09/24 14:03:30 Successfully scanned dependencies
2021/09/24 14:03:30 Launching container with name: acb_step_0
Sending build context to Docker daemon  66.56kB
Step 1/19 : FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210806.v1@sha256:bb94557d668c1cebffb04d0a77aa

In [6]:
print(service.get_logs())

2021-09-24T13:40:16,949962869+00:00 - gunicorn/run 
2021-09-24T13:40:16,949951869+00:00 - iot-server/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-09-24T13:40:16,950000868+00:00 - rsyslog/run 
2021-09-24T13:40:16,951594365+00:00 - nginx/run 
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (14)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 39
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-09-24T13:40:19,109007126+00:00 - iot-server/finish 1 0
2021-09-24T13:40:19,110477021+00:00 - Exit code 1 is normal. Not restarting iot-server.
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-09-24 13:40:19,870 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-09-24 13:40:19,870 | root | INFO | Starting up request id generator
2021-09-24 13:40:19,870 | root | INFO | Star

# Call into your model
Let's check that your echo model deployed successfully. You should be able to do a simple liveness request, as well as a scoring request:

In [7]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
data = {
    "id": "123456789",
    "content": "The quick brown fox jumped over the lazy dog.",
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

{"id": "123456789", "questions": [{"id": "Property Address", "answers": [{"score": 0.44992750883102417, "start": 37, "end": 52, "answer": "910 Elkton, LLC"}, {"score": 0.44992750883102417, "start": 37, "end": 52, "answer": "910 Elkton, LLC"}, {"score": 0.44992750883102417, "start": 37, "end": 52, "answer": "910 Elkton, LLC"}]}, {"id": "Landlord", "answers": [{"score": 0.44992750883102417, "start": 37, "end": 52, "answer": "910 Elkton, LLC"}, {"score": 0.44992750883102417, "start": 37, "end": 52, "answer": "910 Elkton, LLC"}, {"score": 0.44992750883102417, "start": 37, "end": 52, "answer": "910 Elkton, LLC"}]}, {"id": "Tenant", "answers": [{"score": 0.44992750883102417, "start": 37, "end": 52, "answer": "910 Elkton, LLC"}, {"score": 0.44992750883102417, "start": 37, "end": 52, "answer": "910 Elkton, LLC"}, {"score": 0.44992750883102417, "start": 37, "end": 52, "answer": "910 Elkton, LLC"}]}, {"id": "Lease Signed Date", "answers": [{"score": 0.44992750883102417, "start": 37, "end": 52, "